In [93]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [94]:
df = pd.read_csv("USvideos.csv")

In [95]:
data_notNull = df[(df['title'] != '[none]' ) & (df['tags'] != '[none]' )& (df['views'] != '[none]' )& (df['likes'] != '[none]' )&(df['dislikes'] != '[none]' )&(df['comment_count'] != '[none]' )]

C:\Users\abhin\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [96]:
data_notNull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39414 entries, 0 to 40948
Data columns (total 16 columns):
video_id                  39414 non-null object
trending_date             39414 non-null object
title                     39414 non-null object
channel_title             39414 non-null object
category_id               39414 non-null int64
publish_time              39414 non-null object
tags                      39414 non-null object
views                     39414 non-null int64
likes                     39414 non-null int64
dislikes                  39414 non-null int64
comment_count             39414 non-null int64
thumbnail_link            39414 non-null object
comments_disabled         39414 non-null bool
ratings_disabled          39414 non-null bool
video_error_or_removed    39414 non-null bool
description               39119 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 4.3+ MB


In [97]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
common_words =  stopwords.words('english')
common_words = common_words + ['']
def getWords(col:str):
    tagsArray = data_notNull[col]
    tag2D = []
    tagsArray
    for tag in tagsArray:
        lineArray = tag.split('|')
        words = []
        for sentence in lineArray:
            words = words + re.split(r'\W+', sentence.strip())
        tag2D.append(words)
    for i in range(len(tag2D)):
        temp = [u for u in tag2D[i] if u.lower() not in common_words] 
        temp = " ".join(str(x) for x in temp)
        tag2D[i] = temp
    return tag2D

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
import hashlib
#turn all the string in the 2D array into some hash number
def hashArray(myArray):
    result = []
    for words in myArray:
        hashArray = []
        for word in words:
            hashArray.append(abs(hash(word)))
        result.append(hashArray)
    return result

In [99]:
tag_words = getWords('tags')
title_words = getWords('title')

In [100]:
data_notNull['modified_tags'] = tag_words
data_notNull['modified_title'] = title_words
data_notNull
le = LabelEncoder()
le.fit(data_notNull['modified_tags'])
data_notNull['tags_encoded'] = le.transform(data_notNull['modified_tags'])

C:\Users\abhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\abhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\abhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [101]:
le.fit(data_notNull['modified_title'])
data_notNull['title_encoded'] = le.transform(data_notNull['modified_title'])

C:\Users\abhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [102]:
preprocessed_data = data_notNull.drop(['description', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'publish_time', 'channel_title', 'trending_date', 'video_id', 'tags', 'title', 'modified_tags', 'modified_title'], axis=1)
preprocessed_data.head()

,category_id,views,likes,dislikes,comment_count,tags_encoded,title_encoded
0,22,748374,57527,2966,15954,2009,5680
1,24,2418783,97185,6146,12703,4351,5506
2,23,3191434,146033,5339,8181,5085,4364
3,24,343168,10172,666,2146,5184,3920
4,24,2095731,132235,1989,17518,5259,1405


In [103]:
X = preprocessed_data.drop('category_id', axis=1)
y = preprocessed_data['category_id']

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [132]:
#scaling = MinMaxScaler(feature_range=(0, 1)).fit(X_train)
#quantile_scaler = QuantileTransformer(output_distribution='uniform').fit(X_train)
quantile_transformer = QuantileTransformer(output_distribution='uniform').fit(X_train)
#X_train = scaling.transform(X_train)
#X_test = scaling.transform(X_test)
#Xq_train = quantile_scaler.transform(X_train)
#Xq_test = quantile_scaler.transform(X_test)
#Xqn_train = quantile_transformer.transform(X_train)
#Xqn_test = quantile_transformer.transform(X_train)

In [142]:
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=25, random_state=33)
model.fit(X_train, y_train)
#svc = SVC(kernel='rbf', C=1, decision_function_shape='sigmoid', gamma=1000)
#svc.fit(Xqn_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=25,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=33, splitter='best')

In [143]:
y_pred = model.predict(X_test)

In [144]:
accuracy_score(y_test, y_pred)

0.8556387162247875

In [121]:
recall_score(y_test, y_pred, average='macro')

0.8389847493923178

In [56]:
kmodel = KNeighborsClassifier(n_neighbors=32, p=1, weights='distance')
kmodel.fit(Xq_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=32, p=1,
                     weights='distance')

In [59]:
y_predk = kmodel.predict(Xq_test)

In [60]:
accuracy_score(y_test, y_predk)
recall_score(y_test, y_predk)

0.6454212454212455